In [ ]:
# default_exp tensorflow.tflite_converter

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import logging
import logging.handlers
import argparse
import sys
import tensorflow as tf
from mlcore.core import Type
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb
from os.path import join, basename
from google.protobuf import text_format

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

logger = logging.getLogger(__name__)

# Tensorflow Lite Model Converter
> Converts a SavedModel into Tensorflow Lite format. For details, see [Tensorflow Lite Converter](https://www.tensorflow.org/lite/convert)

In [ ]:
# export


def convert_model(saved_model_dir):
    """
    Convert a SavedModel into Tensorflow Lite Format.
    `saved_model_dir`: the path to the SavedModel directory
    returns: the converted Tensorflow Lite model
    """
    logger.info('Converting SavedModel from: {}'.format(saved_model_dir))
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)  # path to the SavedModel directory
    tflite_model = converter.convert()
    return tflite_model

In [ ]:
# export


def save_model(tflite_model, output_path):
    """
    Save a Tensowflow Lite model to disk.
    `tflite_model`: the Tensorflow Lite model
    `output_path`: the folder to save the Tensorflow Lite model
    """
    output_file = join(output_path, 'model.tflite')
    with open(output_file, 'wb') as f:
        f.write(tflite_model)
    logger.info('Successfully save model to file: {}'.format(output_file))

In [ ]:
# export


def add_metadata(categories_file_path, output_path):
    """
    Add meta-data to the Tensowflow Lite model on disk.
    `categories_file_path`: the path to the categories.txt file
    `output_path`: the folder of the Tensorflow Lite model
    """
    # Creates model info.
    model_meta = _metadata_fb.ModelMetadataT()
    model_meta.name = "SSD MobileNetV2"
    model_meta.description = ("Identify which of a known set of objects "
                              "might be present and provide information about their positions "
                              "within the given image or a video stream.")
    model_meta.version = "v1"
    model_meta.author = "Protosolution"
    model_meta.license = ("Apache License. Version 2.0 "
                          "http://www.apache.org/licenses/LICENSE-2.0.")

    # Creates input info.
    input_meta = _metadata_fb.TensorMetadataT()
    input_meta.name = "image"
    input_meta.description = (
        "Input image to be detected. The expected image is {0} x {1}, with "
        "three channels (red, blue, and green) per pixel. Each value in the "
        "tensor is a single byte between 0 and 255.".format(640, 640))
    input_meta.content = _metadata_fb.ContentT()
    input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
    input_meta.content.contentProperties.colorSpace = (
        _metadata_fb.ColorSpaceType.RGB)
    input_meta.content.contentPropertiesType = (
        _metadata_fb.ContentProperties.ImageProperties)
    input_normalization = _metadata_fb.ProcessUnitT()
    input_normalization.optionsType = (
        _metadata_fb.ProcessUnitOptions.NormalizationOptions)
    input_normalization.options = _metadata_fb.NormalizationOptionsT()
    input_normalization.options.mean = [127.5]
    input_normalization.options.std = [127.5]
    input_meta.processUnits = [input_normalization]
    input_meta.stats = _metadata_fb.StatsT()
    input_meta.stats.max = [255]
    input_meta.stats.min = [0]

    # Creates output info.
    output_meta_location = _metadata_fb.TensorMetadataT()
    output_meta_location.name = "location"
    output_meta_location.description = "The locations of the detected boxes."
    output_meta_location.content = _metadata_fb.ContentT()
    output_meta_location.content.content_properties = _metadata_fb.BoundingBoxPropertiesT()
    output_meta_location.content.contentPropertiesType = _metadata_fb.ContentProperties.BoundingBoxProperties
    output_meta_location.content.content_properties.index = [1, 0, 3, 2]
    output_meta_location.content.content_properties.type = _metadata_fb.BoundingBoxType.BOUNDARIES
    output_meta_location.content.range = _metadata_fb.ValueRangeT()
    output_meta_location.content.range.min = 2
    output_meta_location.content.range.max = 2

    output_meta_category = _metadata_fb.TensorMetadataT()
    output_meta_category.name = "category"
    output_meta_category.description = "The categories of the detected boxes."
    output_meta_category.content = _metadata_fb.ContentT()
    output_meta_category.content.content_properties = _metadata_fb.FeaturePropertiesT()
    output_meta_category.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties
    output_meta_category.content.range = _metadata_fb.ValueRangeT()
    output_meta_category.content.range.min = 2
    output_meta_category.content.range.max = 2
    label_file = _metadata_fb.AssociatedFileT()
    label_file.name = basename(categories_file_path)
    label_file.description = "Labels for objects that the model can recognize."
    label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
    output_meta_category.associatedFiles = [label_file]

    output_meta_score = _metadata_fb.TensorMetadataT()
    output_meta_score.name = "score"
    output_meta_score.description = "The scores of the detected boxes."
    output_meta_score.content = _metadata_fb.ContentT()
    output_meta_score.content.content_properties = _metadata_fb.FeaturePropertiesT()
    output_meta_score.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties
    output_meta_score.content.range = _metadata_fb.ValueRangeT()
    output_meta_score.content.range.min = 2
    output_meta_score.content.range.max = 2

    output_meta_detections = _metadata_fb.TensorMetadataT()
    output_meta_detections.name = "number of detections"
    output_meta_detections.description = "The number of the detected boxes."
    output_meta_detections.content = _metadata_fb.ContentT()
    output_meta_detections.content.content_properties = _metadata_fb.FeaturePropertiesT()
    output_meta_detections.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties

    output_groups = _metadata_fb.TensorGroupT()
    output_groups.name = "detection result"
    output_groups.tensorNames = [output_meta_location.name, output_meta_category.name, output_meta_score.name]

    # Creates subgraph info.
    subgraph = _metadata_fb.SubGraphMetadataT()
    subgraph.inputTensorMetadata = [input_meta]
    subgraph.outputTensorMetadata = [output_meta_location, output_meta_category, output_meta_score, output_meta_detections]
    subgraph.outputTensorGroups = [output_groups]
    model_meta.subgraphMetadata = [subgraph]

    b = flatbuffers.Builder(0)
    b.Finish(
        model_meta.Pack(b),
        _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
    metadata_buf = b.Output()

    populator = _metadata.MetadataPopulator.with_model_file(join(output_path, 'model.tflite'))
    populator.load_metadata_buffer(metadata_buf)
    populator.load_associated_files([categories_file_path])
    populator.populate()

## Helper Methods

In [ ]:
# export


def read_pipeline_config(pipeline_config_path):
    """
    Reads the pipeline config file.

    `pipeline_config_path`: The path to the pipeline config file.
    """
    pipeline_config = {}
    with tf.io.gfile.GFile(pipeline_config_path, 'r') as f:
        text_format.Parse(f.read(), pipeline_config)
    return pipeline_config

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    `logging_level`: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)

## Run from command line

To run from command line, use the following command:
`python -m mlcore.tensorflow.tflite_converter [parameters]`

The following parameters are supported:
- `--source`: The path to the folder containing the SavedModel. (e.g.:  *datasets/image_object_detection/car_damage/saved_model*)
- `--output`: The folder to store the Tensorflow Lite model. (e.g.:  *datasets/image_object_detection/car_damage/tflite*)

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    configure_logging()

    parser = argparse.ArgumentParser()
    parser.add_argument("-s",
                        "--source",
                        help="The path to the folder containing the SavedModel.",
                        type=str)
    parser.add_argument("-c",
                        "--categories",
                        help="The categories file to add to the Tensorflow Lite model.",
                        type=str)
    parser.add_argument("-o",
                        "--output",
                        help="The folder to store the Tensorflow Lite model.",
                        type=str)
    args = parser.parse_args()

    save_model(convert_model(args.source), args.output)
    add_metadata(args.categories, args.output)

    logger.info('FINISHED!!!')
